In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
import datetime
import random
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [62]:
def get_sae(year, urg="GEN"):
    # sql = f"select * from sae.bor where id={bor}"
    # df = pd.read_sql(sql, config.connection_string)
    # table = df["table"].iloc[0]
    # column = df["column"].iloc[0]
    # condition = df["condition"].iloc[0]
    sql = f"""
select d.fi, d.passu, p.perso is not null has_pdata, p.etpsal, p.efflib, e.id etab_id, e.rs, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris 
from sae.urgence_detail d
left join sae.urgence_p p on p.fi=d.fi and p.an=d.an and p.perso='M9999'
join etablissement e on e.nofinesset=d.fi
join adresse_raw ar on e.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where d.an={year}
and d.urg='{urg}'
"""
    print(f"Quering SAE for year {year} and urg={urg}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)

In [129]:
def get_iriss():
    sql = f"""
select i.id "iris", i.nom "iris_label", c.code "code_commune", c.nom "commune_label"
from iris.iris i
join iris.commune c on c.id=i.commune_id
"""
    return pd.read_sql(text(sql), config.connection_string)

In [63]:
def get_iris_matrix(time: int, time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m



In [67]:
year=2023
time=60
time_type="HC"
urg="GEN"
year, time, time_type, urg

(2023, 60, 'HC', 'GEN')

In [97]:
# Very long
iris_matrix = get_iris_matrix(time, time_type)
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= 60) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,10010000,10010000,0,0,0,0,10010000
1,10010000,10280000,8,9,10,9,10280000
2,10010000,11880000,6,7,8,7,11880000
3,10010000,13510000,7,8,9,8,13510000
4,10020000,10020000,0,0,0,0,10020000
...,...,...,...,...,...,...,...
772272,2023640000,2022920000,11,26,26,26,2022920000
772273,2023640000,2023640000,0,0,0,0,2023640000
772274,2023650000,2021350000,6,8,9,8,2021350000
772275,2023650000,2023650000,0,0,0,0,2023650000


In [98]:
ps_df = get_sae(year, urg)
ps_df

Quering SAE for year 2023 and urg=GEN

select d.fi, d.passu, p.perso is not null has_pdata, p.etpsal, p.efflib, e.id etab_id, e.rs, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris 
from sae.urgence_detail d
left join sae.urgence_p p on p.fi=d.fi and p.an=d.an and p.perso='M9999'
join etablissement e on e.nofinesset=d.fi
join adresse_raw ar on e.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where d.an=2023
and d.urg='GEN'



,fi,passu,has_pdata,etpsal,efflib,etab_id,rs,dept_id,adresse_norm_id,lon,lat,iris
0,010000024,38394,True,12.52,NaN,186893,CH DE FLEYRIAT,1,102416,5.208596,46.222745,14510102
1,010000032,13853,True,3.83,NaN,186858,CH BUGEY SUD,1,102608,5.684976,45.761208,10340101
2,010005239,22146,True,9.68,0.0,228095,CH DU HAUT BUGEY - GEOVREISSET,1,102683,5.627348,46.275059,12830305
3,010780195,23548,True,0.00,6.0,187194,CLINIQUE CONVERT,1,291,5.253603,46.211526,10530502
4,010780203,38165,True,NaN,11.0,187146,HOPITAL PRIVE D'AMBERIEU,1,102968,5.359558,45.958430,10040201
...,...,...,...,...,...,...,...,...,...,...,...,...
563,130784713,12933,False,NaN,NaN,136221,HP MARSEILLE BEAUREGARD VERT COTEAU,13,84355,5.412129,43.313378,132120301
564,840000343,16524,False,NaN,NaN,140117,CENTRE HOSPITALIER DU PAYS D'APT,84,169627,5.374525,43.869247,840030202
565,780300422,53170,False,NaN,NaN,200723,HOPITAL PRIVE DE L OUEST PARISIEN,78,74699,1.990648,48.778183,786210110
566,150000164,8616,False,NaN,NaN,185396,CH DE MAURIAC,15,112264,2.338774,45.217309,151200000


In [99]:
ps_df["etpsal"] = ps_df["etpsal"].fillna(0)
ps_df["efflib"] = ps_df["efflib"].fillna(0)
ps_df

,fi,passu,has_pdata,etpsal,efflib,etab_id,rs,dept_id,adresse_norm_id,lon,lat,iris
0,010000024,38394,True,12.52,0.0,186893,CH DE FLEYRIAT,1,102416,5.208596,46.222745,14510102
1,010000032,13853,True,3.83,0.0,186858,CH BUGEY SUD,1,102608,5.684976,45.761208,10340101
2,010005239,22146,True,9.68,0.0,228095,CH DU HAUT BUGEY - GEOVREISSET,1,102683,5.627348,46.275059,12830305
3,010780195,23548,True,0.00,6.0,187194,CLINIQUE CONVERT,1,291,5.253603,46.211526,10530502
4,010780203,38165,True,0.00,11.0,187146,HOPITAL PRIVE D'AMBERIEU,1,102968,5.359558,45.958430,10040201
...,...,...,...,...,...,...,...,...,...,...,...,...
563,130784713,12933,False,0.00,0.0,136221,HP MARSEILLE BEAUREGARD VERT COTEAU,13,84355,5.412129,43.313378,132120301
564,840000343,16524,False,0.00,0.0,140117,CENTRE HOSPITALIER DU PAYS D'APT,84,169627,5.374525,43.869247,840030202
565,780300422,53170,False,0.00,0.0,200723,HOPITAL PRIVE DE L OUEST PARISIEN,78,74699,1.990648,48.778183,786210110
566,150000164,8616,False,0.00,0.0,185396,CH DE MAURIAC,15,112264,2.338774,45.217309,151200000


In [100]:
ps_df = ps_df.sort_values(by='iris')
ps_df2 = ps_df.drop_duplicates(subset=['iris'])
ps_df2


,fi,passu,has_pdata,etpsal,efflib,etab_id,rs,dept_id,adresse_norm_id,lon,lat,iris
4,010780203,38165,True,0.00,11.0,187146,HOPITAL PRIVE D'AMBERIEU,1,102968,5.359558,45.958430,10040201
1,010000032,13853,True,3.83,0.0,186858,CH BUGEY SUD,1,102608,5.684976,45.761208,10340101
3,010780195,23548,True,0.00,6.0,187194,CLINIQUE CONVERT,1,291,5.253603,46.211526,10530502
2,010005239,22146,True,9.68,0.0,228095,CH DU HAUT BUGEY - GEOVREISSET,1,102683,5.627348,46.275059,12830305
0,010000024,38394,True,12.52,0.0,186893,CH DE FLEYRIAT,1,102416,5.208596,46.222745,14510102
...,...,...,...,...,...,...,...,...,...,...,...,...
559,950300244,41271,True,0.00,5.0,208675,CHP SAINTE MARIE OSNY,95,62656,2.075184,49.073939,954760102
558,950000364,57898,True,26.30,0.0,208040,HOPITAL NOVO SITE PONTOISE,95,61819,2.094428,49.064959,955000901
560,950300277,23417,True,0.00,8.0,208683,HOPITAL PRIVE NORD PARISIEN,95,62142,2.378056,48.980337,955850123
138,2A0000154,17910,True,6.00,0.0,135483,CLINIQUE DU SUD DE LA CORSE,201,183256,9.207501,41.653334,2012470202


In [101]:
matrix_merge_df = iris_matrix.merge(ps_df2, on="iris", how="left", suffixes=('', ''))
matrix_merge_df["etpsal"] = matrix_merge_df["etpsal"].fillna(0)
matrix_merge_df["efflib"] = matrix_merge_df["efflib"].fillna(0)
matrix_merge_df["passu"] = matrix_merge_df["passu"].fillna(0)
matrix_merge_df[matrix_merge_df["fi"].notnull()]

,iris1,iris2,km,time_hc,time_hp,time,iris,fi,passu,has_pdata,etpsal,efflib,etab_id,rs,dept_id,adresse_norm_id,lon,lat
11,10040201,10040201,0,0,0,0,10040201,010780203,38165.0,True,0.00,11.0,187146.0,HOPITAL PRIVE D'AMBERIEU,1.0,102968.0,5.359558,45.958430
28,10070000,10040201,5,6,7,6,10040201,010780203,38165.0,True,0.00,11.0,187146.0,HOPITAL PRIVE D'AMBERIEU,1.0,102968.0,5.359558,45.958430
148,10340101,10340101,0,0,0,0,10340101,010000032,13853.0,True,3.83,0.0,186858.0,CH BUGEY SUD,1.0,102608.0,5.684976,45.761208
231,10530203,14510102,7,10,13,10,14510102,010000024,38394.0,True,12.52,0.0,186893.0,CH DE FLEYRIAT,1.0,102416.0,5.208596,46.222745
234,10530301,14510102,7,9,11,9,14510102,010000024,38394.0,True,12.52,0.0,186893.0,CH DE FLEYRIAT,1.0,102416.0,5.208596,46.222745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770942,956800112,930720101,7,10,16,10,930720101,930300553,43494.0,True,0.00,9.0,205889.0,CLINIQUE DE L ESTREE,93.0,60484.0,2.367450,48.954903
770970,956800113,930720101,7,10,16,10,930720101,930300553,43494.0,True,0.00,9.0,205889.0,CLINIQUE DE L ESTREE,93.0,60484.0,2.367450,48.954903
770999,956800114,930720101,8,10,16,10,930720101,930300553,43494.0,True,0.00,9.0,205889.0,CLINIQUE DE L ESTREE,93.0,60484.0,2.367450,48.954903
771298,2012470202,2012470202,0,0,0,0,2012470202,2A0000154,17910.0,True,6.00,0.0,135483.0,CLINIQUE DU SUD DE LA CORSE,201.0,183256.0,9.207501,41.653334


In [102]:
# Indices dist-min avec fi non null
idx_ok = matrix_merge_df[matrix_merge_df['fi'].notna()].groupby('iris1')['time'].idxmin()
idx_ok

iris1
10040201          11
10070000          28
10340101         148
10530203         231
10530301         234
               ...  
956800112     770942
956800113     770970
956800114     770999
2012470202    771298
2020500102    771499
Name: time, Length: 6773, dtype: int64

In [104]:
# Indices dist-min sans condition (fallback)
idx_any = matrix_merge_df.groupby('iris1')['time'].idxmin()
idx_any

iris1
10010000           0
10020000           4
10040101           8
10040102           9
10040201          11
               ...  
2023560000    772266
2023610000    772271
2023640000    772273
2023650000    772275
2023660000    772276
Name: time, Length: 48569, dtype: int64

In [105]:
# Construire la sélection finale : idx_ok sinon idx_any
idx_final = idx_ok.combine_first(idx_any)  # Series alignée sur iris
idx_final

iris1
10010000           0.0
10020000           4.0
10040101           8.0
10040102           9.0
10040201          11.0
                ...   
2023560000    772266.0
2023610000    772271.0
2023640000    772273.0
2023650000    772275.0
2023660000    772276.0
Name: time, Length: 48569, dtype: float64

In [106]:
result = matrix_merge_df.loc[idx_final].sort_index().reset_index(drop=True)
result[result["fi"].notnull()]
result

,iris1,iris2,km,time_hc,time_hp,time,iris,fi,passu,has_pdata,etpsal,efflib,etab_id,rs,dept_id,adresse_norm_id,lon,lat
0,10010000,10010000,0,0,0,0,10010000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10020000,10020000,0,0,0,0,10020000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,10040101,10040101,0,0,0,0,10040101,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,10040102,10040102,0,0,0,0,10040102,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,10040201,10040201,0,0,0,0,10040201,010780203,38165.0,True,0.0,11.0,187146.0,HOPITAL PRIVE D'AMBERIEU,1.0,102968.0,5.359558,45.95843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,2023560000,2023560000,0,0,0,0,2023560000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48565,2023610000,2023610000,0,0,0,0,2023610000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48566,2023640000,2023640000,0,0,0,0,2023640000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48567,2023650000,2023650000,0,0,0,0,2023650000,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
dist_df = result[["iris1","km","time_hc","time_hp","fi","passu","has_pdata","etpsal","efflib","rs"]]
dist_df


,iris1,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs
0,10010000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
1,10020000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
2,10040101,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
3,10040102,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
4,10040201,0,0,0,010780203,38165.0,True,0.0,11.0,HOPITAL PRIVE D'AMBERIEU
...,...,...,...,...,...,...,...,...,...,...
48564,2023560000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
48565,2023610000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
48566,2023640000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
48567,2023650000,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN


In [115]:
dist_df2 = dist_df.assign(time_hc=dist_df['time_hc'].where(dist_df['fi'].notna()))
dist_df2 = dist_df2.assign(time_hp=dist_df['time_hp'].where(dist_df['fi'].notna()))
dist_df2 = dist_df2.assign(km=dist_df['km'].where(dist_df['fi'].notna()))
dist_df2[dist_df2["km"].notnull()]

,iris1,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs
4,10040201,0.0,0.0,0.0,010780203,38165.0,True,0.00,11.0,HOPITAL PRIVE D'AMBERIEU
8,10070000,5.0,6.0,7.0,010780203,38165.0,True,0.00,11.0,HOPITAL PRIVE D'AMBERIEU
38,10340101,0.0,0.0,0.0,010000032,13853.0,True,3.83,0.0,CH BUGEY SUD
62,10530203,7.0,10.0,13.0,010000024,38394.0,True,12.52,0.0,CH DE FLEYRIAT
63,10530301,7.0,9.0,11.0,010000024,38394.0,True,12.52,0.0,CH DE FLEYRIAT
...,...,...,...,...,...,...,...,...,...,...
48160,956800112,7.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE
48161,956800113,7.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE
48162,956800114,8.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE
48273,2012470202,0.0,0.0,0.0,2A0000154,17910.0,True,6.00,0.0,CLINIQUE DU SUD DE LA CORSE


In [117]:
dist_df2["tension"] = dist_df2["passu"]/(dist_df2["etpsal"] + dist_df2["efflib"])
dist_df2[dist_df2["km"].notnull()]

,iris1,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs,tension
4,10040201,0.0,0.0,0.0,010780203,38165.0,True,0.00,11.0,HOPITAL PRIVE D'AMBERIEU,3469.545455
8,10070000,5.0,6.0,7.0,010780203,38165.0,True,0.00,11.0,HOPITAL PRIVE D'AMBERIEU,3469.545455
38,10340101,0.0,0.0,0.0,010000032,13853.0,True,3.83,0.0,CH BUGEY SUD,3616.971279
62,10530203,7.0,10.0,13.0,010000024,38394.0,True,12.52,0.0,CH DE FLEYRIAT,3066.613419
63,10530301,7.0,9.0,11.0,010000024,38394.0,True,12.52,0.0,CH DE FLEYRIAT,3066.613419
...,...,...,...,...,...,...,...,...,...,...,...
48160,956800112,7.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE,4832.666667
48161,956800113,7.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE,4832.666667
48162,956800114,8.0,10.0,16.0,930300553,43494.0,True,0.00,9.0,CLINIQUE DE L ESTREE,4832.666667
48273,2012470202,0.0,0.0,0.0,2A0000154,17910.0,True,6.00,0.0,CLINIQUE DU SUD DE LA CORSE,2985.000000


In [131]:
iriss = get_iriss()
iriss

,iris,iris_label,code_commune,commune_label
0,821300000,Montjoi,82130,Montjoi
1,811540000,Marnaves,81154,Marnaves
2,840060000,Auribeau,84006,Auribeau
3,821570000,Saint-Beauzeil,82157,Saint-Beauzeil
4,840880101,Secteur Ouest,84088,Pernes-les-Fontaines
...,...,...,...,...
48564,912280114,Mousseau-Bonhomme en Pierre,91228,Évry-Courcouronnes
48565,912280204,Courcouronnes Centre Nord,91228,Évry-Courcouronnes
48566,920460108,Les Sablonnières,92046,Malakoff
48567,940460202,Alfort 2,94046,Maisons-Alfort


In [132]:
dist_df2["iris"] = dist_df2["iris1"]
dist_df3 = dist_df2.merge(iriss, on="iris", how="left", suffixes=('', ''))
dist_df3

,iris1,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs,tension,fake_time,iris,iris_label,code_commune,commune_label
0,10010000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,10010000,L'Abergement-Clémenciat,01001,L'Abergement-Clémenciat
1,10020000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,10020000,L'Abergement-de-Varey,01002,L'Abergement-de-Varey
2,10040101,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,10040101,Les Pérouses-Triangle d'Activités,01004,Ambérieu-en-Bugey
3,10040102,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,10040102,Longeray-Gare,01004,Ambérieu-en-Bugey
4,10040201,0.0,0.0,0.0,010780203,38165.0,True,0.0,11.0,HOPITAL PRIVE D'AMBERIEU,3469.545455,0.0,10040201,Centre-Saint-Germain-Vareilles,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,2023560000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,2023560000,Zalana,2B356,Zalana
48565,2023610000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,2023610000,Zilia,2B361,Zilia
48566,2023640000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,2023640000,Zuani,2B364,Zuani
48567,2023650000,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,61.0,2023650000,San-Gavino-di-Fiumorbo,2B365,San-Gavino-di-Fiumorbo


In [134]:
dist_df3["fake_time"] = dist_df3["time_hc"]
dist_df3["fake_time"] = dist_df3["fake_time"].fillna(61)
dist_df3["fake_time"].describe()

count    48569.000000
mean        53.743849
std         18.071615
min          0.000000
25%         61.000000
50%         61.000000
75%         61.000000
max         61.000000
Name: fake_time, dtype: float64

In [135]:
final = dist_df3.drop(["iris1", "fake_time"], axis=1)
final

,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs,tension,iris,iris_label,code_commune,commune_label
0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,10010000,L'Abergement-Clémenciat,01001,L'Abergement-Clémenciat
1,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,10020000,L'Abergement-de-Varey,01002,L'Abergement-de-Varey
2,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,10040101,Les Pérouses-Triangle d'Activités,01004,Ambérieu-en-Bugey
3,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,10040102,Longeray-Gare,01004,Ambérieu-en-Bugey
4,0.0,0.0,0.0,010780203,38165.0,True,0.0,11.0,HOPITAL PRIVE D'AMBERIEU,3469.545455,10040201,Centre-Saint-Germain-Vareilles,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,2023560000,Zalana,2B356,Zalana
48565,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,2023610000,Zilia,2B361,Zilia
48566,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,2023640000,Zuani,2B364,Zuani
48567,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,2023650000,San-Gavino-di-Fiumorbo,2B365,San-Gavino-di-Fiumorbo


In [136]:
final[final["iris"]==381850212]

,km,time_hc,time_hp,fi,passu,has_pdata,etpsal,efflib,rs,tension,iris,iris_label,code_commune,commune_label
17698,6.0,9.0,11.0,380785956,23298.0,True,0.0,10.0,CLINIQUE DES CEDRES,2329.8,381850212,Préfecture,38185,Grenoble
